In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Libraries :

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import time
import random
from collections import Counter
import matplotlib.pyplot as plt

# Class *Instabot* :

In [ ]:
class InstaBot :

    def __clearField(self, field) :
        """
        Clears the text-fields provided as an argument
        """
        field.send_keys(Keys.CONTROL + "a")
        field.send_keys(Keys.DELETE)
    
    def __waitRandomShortDuration(self, minSeconds = 1, maxSeconds = 3) :
        """
        Makes the Web Browser sleeps for 1-3 seconds randomly 
        """
        if minSeconds >= maxSeconds :
            maxSeconds = minSeconds + 2
        time.sleep(random.randint(minSeconds, maxSeconds))
    
    def __init__(self) :
        """
        Initializes the Web Browser, logs-in to the provided credentials in the file "credentials.txt"
        """
        self.credentialsFileName = 'credentials.txt'
        self.driver = webdriver.Firefox(executable_path = 'geckodriver')
        self.driver.maximize_window()
        self.driverWait = WebDriverWait(self.driver, 10)
        
        with open(self.credentialsFileName) as credentialsObj :
            credentials = credentialsObj.readlines()
            username = credentials[0].strip()
            password = credentials[1].strip()
        
        self.driver.get('https://www.instagram.com/')
        
        try :
            usernameBox = self.driverWait.until(EC.presence_of_element_located((By.NAME, 'username')))
            passwordBox = self.driverWait.until(EC.presence_of_element_located((By.NAME, 'password')))

            usernameBox.click()
            self.__clearField(usernameBox)
            usernameBox.send_keys(username)
            self.__waitRandomShortDuration()

            passwordBox.click()
            self.__clearField(passwordBox)
            passwordBox.send_keys(password)
            self.__waitRandomShortDuration()

            loginButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[text() = "Log In"]//ancestor::button')))
            loginButton.click()
        except :
            pass
        
        try :
            self.__waitRandomShortDuration()
            notNowButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Not Now"]')))
            notNowButton.click()
        except :
            pass
        
        try :
            self.__waitRandomShortDuration()
            notificationNotNowButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Not Now"]')))
            notificationNotNowButton.click()
        except :
            pass
    
    def __returnToHomepage(self) :
        """
        Makes the Web Browser returns to the main Instagram Wall of the user
        """
        self.driver.refresh()
        instagramButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "cq2ai"]//img')))
        instagramButton.click()
        self.__waitRandomShortDuration()
    
    def __getToTheSearchBox(self) :
        """
        Brings to the Search Bar of homepage and returns the search box element
        """
        self.__returnToHomepage()
        
        searchPortion = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//span[text() = "Search"]')))
        searchPortion.click()
        
        try :
            clearAllSearches = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Clear All"]')))
            clearAllSearches.click()
            self.__waitRandomShortDuration()
            
            clearAllButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "mt3GC"]//button[text() = "Clear All"]')))
            clearAllButton.click()
            self.__waitRandomShortDuration()
        except :
            pass
        
        searchPortion = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "LWmhU _0aCwM"]')))
        searchPortion.click()
        searchBox = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "LWmhU _0aCwM"]/input')))
        return searchBox
    
    def __loadAllScrollableContent(self, element) :
        """
        Scroll through all the contents till no more data is left to be loaded for any element passed as the parameter
        """
        currentScrollHeight = self.driver.execute_script('return arguments[0].scrollHeight;', element)
        while True :
            self.driver.execute_script('arguments[0].scrollTo(0, arguments[1]);', element, currentScrollHeight)
            self.__waitRandomShortDuration(minSeconds = 2)
            newScrollHeight = self.driver.execute_script('return arguments[0].scrollHeight;', element)
            if currentScrollHeight == newScrollHeight :
                break
            currentScrollHeight = newScrollHeight
    
    def __failedOperationMessage(self) :
        """
        Just an error message if any functionality fails
        """
        print('!!Operation Failed due to slow Internet or Webdriver Failure!!')
    
    def __getToHandleOrHashtagWall(self, handleOrHashtagName) :
        """
        Given a correct handle name or a hashtag (Case - Sensitive), enter the handle's Instagram wall.
        """
        searchBox = self.__getToTheSearchBox()
        searchBox.send_keys(handleOrHashtagName)
        self.__waitRandomShortDuration()
        
        searchResults = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "_01UL2"]')))
        self.__loadAllScrollableContent(searchResults)
        
        allHandlesOrHashtags = self.driverWait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class = "_01UL2"]//div[starts-with(@class, "fuqBx")]/div/a')))
        for handleOrHashtag in allHandlesOrHashtags :
            if handleOrHashtag.find_element_by_css_selector('div > div + div > div').text == handleOrHashtagName :
                handleOrHashtag.click()
                self.__waitRandomShortDuration()
                return
        print('Insta Handle Not Found!')
    
    def __likeCurrentPost(self) :
        """
        Given that the page to be liked is loaded, true represents that the post is liked, false represents that the post is already liked.
        """
        likeButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//article//div[starts-with(@class, "eo2As")]/section//button')))
        if likeButton.find_element_by_tag_name('svg').get_attribute('aria-label') == 'Unlike' :
            return False
        likeButton.click()
        self.__waitRandomShortDuration()
        return True
    
    def __unlikeCurrentPost(self) :
        """
        Given that the page to be unliked is loaded, true represents that the post in unliked, false represents that the post is already not liked.
        """
        likeButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//article//div[starts-with(@class, "eo2As")]/section//button')))
        if likeButton.find_element_by_tag_name('svg').get_attribute('aria-label') == 'Like' :
            return False
        likeButton.click()
        self.__waitRandomShortDuration()
        return True
    
    def __like_unlike_First_n_postsOfHandle(self, handleName, numPosts, like = True) :
        """
        Likes / Unlikes the given num of posts of handlename
        """
        try :
            self.__getToHandleOrHashtagWall(handleName)
            postButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//span[text() = "Posts"]')))
            postButton.click()
            self.__waitRandomShortDuration()
            
            postsArea = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//article[@class = "ySN3v"]')))
            try :
                self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//article[@class = "ySN3v"]//h2')))
                print("Account Private! Can't access posts!")
            except:
                try :
                    currentMousePointer = ActionChains(self.driver)
                    postsArea = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//article[@class = "ySN3v"]')))
                    postButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//span[text() = "Posts"]')))
                    currentMousePointer.move_to_element(postButton)
                    self.__waitRandomShortDuration()
                    currentMousePointer.click()
                    currentMousePointer.perform()

                    firstPost = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//article//a//img')))
                    currentMousePointer.click(firstPost)
                    currentMousePointer.perform()

                    for postNo in range(numPosts) :
                        self.__waitRandomShortDuration(maxSeconds = 5)
                        if like :
                            if not self.__likeCurrentPost() :
                                print('Post #' + str(postNo + 1) + ", is already liked!")
                        else :
                            if not self.__unlikeCurrentPost() :
                                print('Post #' + str(postNo + 1) + ", is not liked already!")
                        try :
                            nextButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//a[contains(@class, "coreSpriteRightPaginationArrow")]')))
                            nextButton.click()
                        except :
                            break

                    closeButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//body/div[last()]/div[last()]//button')))
                    closeButton.click()
                        
                except :
                    print("No posts available for the user!")
        except :
            self.__failedOperationMessage()
    
    def __commentCurrentPost(self, comment) :
        """
        Comments the (comment) on the current post
        """
        currentMousePointer = ActionChains(self.driver)
        commentArea = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//form[@class = "X7cDz"]')))
        currentMousePointer.move_to_element(commentArea)
        currentMousePointer.double_click()
        currentMousePointer.perform()
        self.__waitRandomShortDuration()
        
        commentBox = commentArea.find_element_by_tag_name('textarea')
        self.__clearField(commentBox)
        commentBox.send_keys(comment)
        commentBox.submit()
    
    def __preprocessHashtag(self, hashtagName) :
        """
        Will transform the hashtag query in the format fit for Instagram
        """
        hashtagName = hashtagName.lower()
        if not hashtagName.startswith('#') :
            hashtagName = '#' + hashtagName
        return hashtagName
    
    def __getToHashtagFirstPost(self, hashtagName) :
        """
        Will lead the WebDriver to page of the first post of the input hashtag
        """
        hashtagName = self.__preprocessHashtag(hashtagName)
        self.__getToHandleOrHashtagWall(hashtagName)
        
        currentMousePointer = ActionChains(self.driver)
            
        postsArea = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//article[@class = "KC1QD"]')))
        firstPost = postsArea.find_element_by_tag_name('img')
        self.__waitRandomShortDuration()

        currentMousePointer.click(firstPost)
        currentMousePointer.perform()
    
    def __updateTrendingHashtags(self, hashtagsWithFrequency) :
        """
        Merges the trending hashtags with the old hashtag list.
        """
        try :
            hashtagInfo = pd.read_csv('hashtags_logs.csv')
        except :
            pd.DataFrame([], columns = ['Hashtag', 'Count']).to_csv('hashtags_logs.csv', index = False)
            hashtagInfo = pd.read_csv('hashtags_logs.csv')
        
        columnNames = hashtagInfo.columns
        hashtagData = Counter()
        for index in range(hashtagInfo.shape[0]) :
            name = hashtagInfo.iloc[index][columnNames[0]]
            count = hashtagInfo.iloc[index][columnNames[1]]
            hashtagData[name] += count
        
        for name, count in hashtagsWithFrequency :
            hashtagData[name] += count
        
        orderedData = []
        for name, count in hashtagData.most_common() :
            orderedData.append([name, count])
        
        pd.DataFrame(orderedData, columns = columnNames).to_csv('hashtags_logs.csv', index = False)


    # Main Functionality #1
    def getFirst_n_searchHandles(self, query, maxHandles = 20) :
        """
        Returns the first (maxHandles) number of posts instagram handles on topic query
        """
        try :
            searchBox = self.__getToTheSearchBox()
            searchBox.send_keys(query)
            self.__waitRandomShortDuration()

            searchResults = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "_01UL2"]')))
            self.__loadAllScrollableContent(searchResults)
            allPosts = self.driverWait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class = "_01UL2"]//div[starts-with(@class, "fuqBx")]/div/a')))

            handles = []
            for post in allPosts:
                if len(handles) >= maxHandles :
                    break
                if post.find_elements_by_css_selector('div > div > div > canvas') :
                    handles.append(post.find_element_by_css_selector('div > div + div > div').text)

            return handles
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #2
    def followHandle(self, handleName) :
        """
        Searches the handle and follows it, only if the handle exists and that handle is not yet followed
        """
        try :
            self.__getToHandleOrHashtagWall(handleName)
            followButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@id="react-root"]/section/main/div/header/section/div/div//button')))
            if followButton.text == 'Follow' :
                self.__waitRandomShortDuration()
                followButton.click()
                print('Followed / Requested!')
            elif followButton.text == 'Requested':
                print('Already Requested!')
            else :
                print('Already Following!')
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #3
    def unfollowHandle(self, handleName) :
        """
        Searches the handle and unfollows it, only if the handle exists and that handle is not yet followed
        """
        try :
            self.__getToHandleOrHashtagWall(handleName)
            messageButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@id="react-root"]/section/main/div/header/section/div/div//button')))
            if messageButton.text == 'Message' :
                unfollowButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@id="react-root"]/section/main/div/header/section/div[1]/div[2]/div/div[2]/div/span/span[1]/button')))
                unfollowButton.click()
                self.__waitRandomShortDuration()

                finalUnfollowButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Unfollow"]')))
                finalUnfollowButton.click()
                print('Unfollowed!')
            elif messageButton.text == 'Requested' :
                unfollowButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Requested"]')))
                unfollowButton.click()
                self.__waitRandomShortDuration()

                finalUnfollowButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Unfollow"]')))
                finalUnfollowButton.click()
                print('Unfollowed!')
            else :
                print('Not Following!')
        except :
            self.__failedOperationMessage()
    
    
    # Main Functionality #4
    def likeFirst_n_postsOfHandle(self, handleName, numPosts = 15) :
        """
        Likes the first n posts of the given handle, default values is 15 posts
        """
        self.__like_unlike_First_n_postsOfHandle(handleName, numPosts)
    
    # Main Functionality #5
    def unlikeFirst_n_postsOfHandle(self, handleName, numPosts = 15) :
        """
        Unlikes the first n posts of the given handle, default values is 15 posts
        """
        self.__like_unlike_First_n_postsOfHandle(handleName, numPosts, like = False)
    
    # Main Functionality #6
    def checkStoriesOfHandle(self, handleName) :
        """
        Checks the stories of a given Instagram Handle, if story is available.
        """
        try :
            self.__getToHandleOrHashtagWall(handleName)
            
            storyArea = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//canvas[@class = "CfWVH"]/..')))
            if storyArea.get_attribute('aria-disabled') == 'true' :
                print('No Stories Available for the Handle!')
            else :
                if storyArea.find_element_by_tag_name('canvas').get_attribute('height') == '166' :
                    print('All available stories are watched! Playing again!')
                
                storyArea = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//canvas[@class = "CfWVH"]/..')))
                storyArea.click()
                self.__waitRandomShortDuration()
                
                storyWait = WebDriverWait(self.driver, 5)
                while True :
                    try :
                        storyWait.until(EC.presence_of_element_located((By.XPATH, '//div[@id = "react-root"]/section/div/div')))
                        self.__waitRandomShortDuration(minSeconds = 5)
                        continue
                    except :
                        break
                    
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #7
    def getFirst_n_hashtags(self, hashtagName, numHashtags = 5) :
        """
        Gets the first top (numHashtags) hashtags. The default value of number of hashtags is 5
        """
        try :
            searchBox = self.__getToTheSearchBox()
            
            hashtagName = self.__preprocessHashtag(hashtagName)
            searchBox.send_keys(hashtagName)
            self.__waitRandomShortDuration()

            searchResults = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "_01UL2"]')))
            self.__loadAllScrollableContent(searchResults)
            allHashtags = self.driverWait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class = "_01UL2"]//div[starts-with(@class, "fuqBx")]/div/a')))

            hashtags = []
            for handle in allHashtags:
                if len(hashtags) >= numHashtags :
                    break
                if handle.find_elements_by_css_selector('div > div > div > svg') and handle.find_element_by_css_selector('div > div > div > svg').get_attribute('aria-label') == 'Hashtag':
                    hashtags.append(handle.find_element_by_css_selector('div > div + div > div').text)

            
            return hashtags
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #8
    def followHashtag(self, hashtagName) :
        """
        Follows a specified hashtag if that hashtag shows up in the search
        """
        try :
            hashtagName = self.__preprocessHashtag(hashtagName)
            self.__getToHandleOrHashtagWall(hashtagName)
            
            try :
                self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Following"]')))
                print('Already following this hashtag!')
            except :
                followButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Follow"]')))
                followButton.click()
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #9
    def unfollowHashtag(self, hashtagName) :
        """
        Unfollows a specified hashtag if that hashtag shows up in the search
        """
        try :
            hashtagName = self.__preprocessHashtag(hashtagName)
            self.__getToHandleOrHashtagWall(hashtagName)
            
            try :
                followingButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//button[text() = "Following"]')))
                followingButton.click()
            except :
                print('Not yet following the hashtag!')
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #10
    def findTop_n_trendingHashtagsFor_a_hashtag_and_visualize(self, hashtagName, n = 10, maxPosts = 75) :
        """
        Finds the first top (n default value 10) most occuring hashtags from continous number of (maxPosts default value 75) for the input hashtagName
        """
        try :
            hashtagName = self.__preprocessHashtag(hashtagName)
            self.__getToHashtagFirstPost(hashtagName)

            hashtagsFrequency = Counter()
            totalAvailablePosts = 0
            for postNo in range(maxPosts) :
                self.__waitRandomShortDuration()
                try :
                    postRelatedHashtags = [content.text.lower() for content in self.driverWait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'xil3i')))]
                except :
                    postRelatedHashtags = []

                for hashtag in postRelatedHashtags :
                    if hashtag.lower() == hashtagName :
                        continue
                    hashtagsFrequency[hashtag.lower()] += 1

                totalAvailablePosts += 1
                try :
                    nextButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//a[contains(@class, "coreSpriteRightPaginationArrow")]')))
                    nextButton.click()
                except :
                    break

            closeButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//body/div[last()]/div[last()]//button')))
            closeButton.click()

            topMostHashtagsWithFrequencies = hashtagsFrequency.most_common()[: n]
            topHashTags = np.array(topMostHashtagsWithFrequencies)
            plt.pie(topHashTags[:, 1], labels = topHashTags[:, 0], autopct = "%.2f%%", explode = [0.1 for _ in range(topHashTags.shape[0])])
            plt.title('Percentage Distribution of top ' + str(n) + ' hashtags for the topic ' + hashtagName + ' for recent ' + str(totalAvailablePosts) + ' posts')
            plt.axis('equal')
            plt.show()

            choice = input('To update hashtags Enter "Y", else enter anything else : ')
            if choice.lower().strip() == 'y' :
                self.__updateTrendingHashtags(topMostHashtagsWithFrequencies)
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #11
    def like_n_continousPostsByHashtag(self, hashtagName, n = 15) :
        """
        Likes top (n default value 15) continous posts
        """
        try :
            hashtagName = self.__preprocessHashtag(hashtagName)
            self.__getToHashtagFirstPost(hashtagName)
            
            for postNo in range(n) :
                self.__waitRandomShortDuration()
                self.__likeCurrentPost()
                
                try :
                    nextButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//a[contains(@class, "coreSpriteRightPaginationArrow")]')))
                    nextButton.click()
                except :
                    break
            
            closeButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//body/div[last()]/div[last()]//button')))
            closeButton.click()
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #12
    def follow_n_randomHandles(self, hashtagName, n = 5) :
        """
        follow (n default value 5) random handles according to a hashtag
        """
        try :
            hashtagName = self.__preprocessHashtag(hashtagName)
            self.__getToHashtagFirstPost(hashtagName)

            handlesFollowed = []
            while len(handlesFollowed) < n :
                self.__waitRandomShortDuration()

                handleArea = self.driverWait.until(EC.presence_of_element_located((By.XPATH, "//article/header/div[2]")))

                followButton = handleArea.find_element_by_tag_name('button')
                if followButton.text == 'Following' :
                    pass
                else :
                    if random.choice([True, False]) :
                        followButton.click()
                        handlesFollowed.append(handleArea.find_element_by_tag_name('a').text)
                        self.__waitRandomShortDuration()

                try :
                    nextButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//a[contains(@class, "coreSpriteRightPaginationArrow")]')))
                    nextButton.click()
                except :
                    break

            closeButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//body/div[last()]/div[last()]//button')))
            closeButton.click()

            return handlesFollowed
        
        except :
            self.__failedOperationMessage()
    
    # Main Functionality #13
    def getCurrentFollowersAndFollowing(self) :
        """
        Returns values of current number of followers and following respectively as a tuple
        """
        try :
            self.__returnToHomepage()
            
            buttons = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "_47KiJ"]/div[position() = last()]')))
            buttons.click()
            self.__waitRandomShortDuration()
            
            profileButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[text() = "Profile"]')))
            profileButton.click()
            
            profileInfo = self.driverWait.until(EC.presence_of_element_located((By.CLASS_NAME, 'k9GMp')))
            contents = profileInfo.find_elements_by_css_selector('li > a > span')
            
            return (int(contents[0].text), int(contents[1].text), )
        except :
                self.__failedOperationMessage()
    
    def __del__(self) :
        """
        A destructor used to first logout of the Instagram Account and then closes the browser
        """
        try :
            self.__returnToHomepage()

            buttons = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "_47KiJ"]/div[position() = last()]')))
            buttons.click()
            self.__waitRandomShortDuration()

            logoutButton = self.driverWait.until(EC.presence_of_element_located((By.XPATH, '//div[@class = "-qQT3"][2]')))
            logoutButton.click()
            self.__waitRandomShortDuration()
        
        except :
            pass
        self.driver.close()

# Supporter Functions :

In [ ]:
def generalActivities(instaBotObject) :
    try :
        while True :
            print('\nGeneral Activities Menu :\n')
            print('Enter 1, to get first number of instagram handles according to search query')
            print('Enter 2, to follow a instagram handle')
            print('Enter 3, to unfollow a instagram handle')
            print('Enter 4, to check story of a handle')

            choice = input('Enter anything else to go back to Main Menu : ')

            if choice.strip() == '1' :
                query = input('Enter search query : ')
                limitChange = input('Default limit of results is 20, Enter "Y" to change the limit, or Enter anything else to comply with the default value : ')
                if limitChange.lower() == 'y' :
                    maxPosts = int(input('Enter limit : '))
                    ans = instaBotObject.getFirst_n_searchHandles(query, maxPosts)
                else :
                    ans = instaBotObject.getFirst_n_searchHandles(query)
                print("\n(OUTPUT):The results are :", ans)

            elif choice.strip() == '2' :
                query = input('Enter insta handle to follow (Case-Sensitive) : ')
                instaBotObject.followHandle(query)

            elif choice.strip() == '3' :
                query = input('Enter insta handle to unfollow (Case-Sensitive) : ')
                instaBotObject.unfollowHandle(query)

            elif choice.strip() == '4' :
                query = input('Enter insta handle to check stories (Case-Sensitive) : ')
                instaBotObject.checkStoriesOfHandle(query)

            else :
                break

    except :
        print('!!Invalid Input!!')
        print('Going back to Main Menu')


def handleRealtedActivities(instaBotObject) :
    try :
        while True :
            print('\nHandle Related Activities Menu :\n')
            print('Enter 1, Like some number of recent posts for a handle')
            print('Enter 2, Unlike some number of recent posts for a handle')

            choice = input('Enter anyhting else to ga back to Blogging Related Activities Menu :')

            if choice.strip() == '1' :
                query = input('Enter insta handle to like posts (Case-Sensitive) : ')
                limitChange = input('Default Limit of posts is 15, enter "Y" to change that limit, or Enter anything else to comply with the default value : ')
                if limitChange.lower().strip() == 'y' :
                    maxPosts = int(input('Enter limit : '))
                    instaBotObject.likeFirst_n_postsOfHandle(query, maxPosts)
                else :
                    instaBotObject.likeFirst_n_postsOfHandle(query)

            elif choice.strip() == '2' :
                query = input('Enter insta handle to unlike posts (Case-Sensitive) : ')
                limitChange = input('Default Limit of posts is 15, enter "Y" to change that limit, or Enter anything else to comply with the default value : ')
                if limitChange.lower().strip() == 'y' :
                    maxPosts = int(input('Enter limit : '))
                    instaBotObject.unlikeFirst_n_postsOfHandle(query, maxPosts)
                else :
                    instaBotObject.unlikeFirst_n_postsOfHandle(query)

            else :
                break
    except :
        print('!!Invalid Input!!')
        print('Going back to Blogging Related Activities Menu')

def hashtagRelatedActivities(instaBotObject) :
    try :
        while True :
            print('\nHashtags Related Activities Menu :\n')
            print('Enter 1, to find some number of related hashtags according to query')
            print('Enter 2, to follow a hashtag')
            print('Enter 3, to unfollow a hashtag')
            print('Enter 4, to like some continous posts of a hashtag')
            print('Enter 5, to follow some random handles according to a hashtag')
            
            choice = input('Enter anyhting else to ga back to Blogging Related Activities Menu :')
            
            if choice.strip() == '1':
                query = input('Enter hashtag to search (Case-Sensitive) : ')
                limitChange = input('Default Limit of posts is 5, enter "Y" to change that limit, or Enter anything else to comply with the default value : ')
                if limitChange.lower().strip() == 'y' :
                    maxHashtags = int(input('Enter limit : '))
                    ans = instaBotObject.getFirst_n_hashtags(query, maxHashtags)
                else :
                    ans = instaBotObject.getFirst_n_hashtags(query)
                print('"\n(OUTPUT):The list of hashtags are :', ans)
            
            elif choice.strip() == '2' :
                query = input('Enter hashtag to follow : ')
                instaBotObject.followHashtag(query)
            
            elif choice.strip() == '3' :
                query = input('Enter hashtag to unfollow : ')
                instaBotObject.unfollowHashtag(query)
            
            elif choice.strip() == '4' :
                query = input('Enter hashtag name : ')
                limitChange = input('Default Limit of posts is 15, enter "Y" to change that limit, or Enter anything else to comply with the default value : ')
                if limitChange.lower().strip() == 'y' :
                    maxPosts = int(input('Enter Limit : '))
                    instaBotObject.like_n_continousPostsByHashtag(query, maxPosts)
                else :
                    instaBotObject.like_n_continousPostsByHashtag(query)
            
            elif choice.strip() == '5' :
                query = input('Enter hashtag name : ')
                limitChange = input('Default Limit of handles that can be followed  is 5, enter "Y" to change that limit, or Enter anything else to comply with the default value : ')
                if limitChange.lower().strip() == 'y' :
                    maxHandles = int(input('Enter Limit : '))
                    instaBotObject.follow_n_randomHandles(query, maxHandles)
                else :
                    instaBotObject.follow_n_randomHandles(query)
            
            else :
                break
    except :
        print('!!Invalid Input!!')
        print('Going back to Blogging Related Activities Menu')

def bloggingActivities(instaBotObject) :
    try :
        while True :
            print('\nBlogging Related Activities Menu :\n')
            print('Enter 1, for Handle Related Activities')
            print('Enter 2, for Hashtag Related Activities')

            choice = input('Enter anything else to go back to Main Menu : ')

            if choice.strip() == '1' :
                handleRealtedActivities(instaBotObject)
            elif choice.strip() == '2':
                hashtagRelatedActivities(instaBotObject)
            else :
                break
    except :
        print('!!Invalid Input!!')
        print('Going back to Main Menu')

def statisticsRelatedActivities(instaBotObject) :
    try :
        while True :
            print('\nStatistic and Analytics related Activities Menu :\n')
            print('Enter 1, to view current followers and following')
            print('Enter 2, to browse through posts of a hashtag, look for some number of trending hashtags around a given query hashtag')
            
            choice = input('Enter anything else to go back to Main Menu : ')
            
            if choice.strip() == '1' :
                ans = instaBotObject.getCurrentFollowersAndFollowing()
                print('"\n(OUTPUT):')
                print('Followers :', ans[0])
                print('Following :', ans[1])
            
            elif choice.strip() == '2' :
                query = input('Enter hashtag : ')
                
                n = 10
                topHashtagsLimitChange = input('The default value of top hashtags to extract is 10, Enter "Y", to change the limit, or Enter anything else to comply with the default value : ')
                if topHashtagsLimitChange.lower().strip() == 'y' :
                    n = int(input('Enter Limit : '))
                
                maxPosts = 75
                maxPostsLimitChange = input('The default value is to browse through 75 posts, Enter "Y", to change the limit, or Enter anything else to comply with the default value : ')
                if maxPostsLimitChange.lower().strip() == 'y' :
                    maxPosts = int(input('Enter Limit : '))
                instaBotObject.findTop_n_trendingHashtagsFor_a_hashtag_and_visualize(query, n, maxPosts)
            
            else :
                break
    except :
        print('!!Invalid Input!!')
        print('Going back to Main Menu')

def mainMenu(instaBotObject) :
    try :
        while True :
            print('\nMain Menu : \n')
            print('Enter 1, for General Activities')
            print('Enter 2, for Blogging Activities')
            print('Enter 3, for Statistics and Analytics Activities')
            
            choice = input('Enter anything else to exit : ')
            
            if choice.strip() == '1' :
                generalActivities(instaBotObject)
            elif choice.strip() == '2' :
                bloggingActivities(instaBotObject)
            elif choice.strip() == '3' :
                statisticsRelatedActivities(instaBotObject)
            else :
                break
    except :
        pass
    del instaBotObject

# Main Application :

In [ ]:
while True :
    print('\nEnter 1, to start Bot\n')
    
    choice = input('Enter anything else to end bot : ')
    
    if choice.strip() == '1' :
        mainMenu(InstaBot())
    else :
        break